In [4]:
import numpy as np
from matplotlib import pyplot as plt
import warnings
import matplotlib.cm as cm
%matplotlib inline
import random 

import pandas as pd
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
#from scipy.sparse.linalg import lsqr

from sklearn.model_selection import train_test_split

In [5]:
def predict(data, branch, name):
    data=data.reset_index(drop=True)
    data[name]=np.zeros(len(data))
    for j in range(len(data)):
        p=data.loc[j, :]
        if p[branch[0][0][0]]<branch[0][0][1]:
            sep=[0] 
        else:
            sep=[1]
        val='bol'
        for i in range(1, len(branch[1:])):
            n=0           
            for k in range(len(sep)):
                if sep[k]==1:
                    n+=2**(i-k-1)
            if len(branch[i][n])==4:
                val=branch[i][n][3]
                break ##Added to try and sort out code not working. If this above condition is made we have reached 
                # the leaf and need to not try and travel further down the tree!
            elif p[branch[i][n][0]]<branch[i][n][1]:
                sep.append(0)
            else:
                sep.append(1)
        if val=='bol':
            n=0           
            for k in range(len(sep)):
                if sep[k]==1:
                    n+=2**(len(sep)-k-1)
            val=branch[-1][n][3]
        data.at[j, name]=val
    return data

Hello Denise and Douglas!

I'm a 24 old year old from Newcastle hoping to travel around Scotland by bike this Autumn. I'm an adventurous, outdoorsie person and I really love the wilder side of Scotland, I've taken a couple of years off since I graduated from University and am really keen to do this trip before finding a job in the new Year. 

Your Inn sounds like a welcoming and totally unique place. The history of it is very interesting and I love that you try and eat meals with the guests. 

I think

In [6]:
def rf_trees(data, target, min_data, max_branches, max_features):
    print('I hope youve inputted the max_features option as a decimal percentage')
    n=0
    leaf=[]
    v, col, sp, s1, s2=split_rf(data, target, min_data, max_features)
    f=[[col, sp, 0]]
    d =  [[[0], s1], [[1], s2]]
    while n < max_branches:
        n += 1
        new=[]
        print('the size of d is {}'.format(len(d)))
        for i in d:
            try:
                v, col, sp, s1, s2=split_rf(i[1], target, min_data, max_features)
                t=i[0].copy()
                p=i[0].copy()
                if col == 0: #The idea of this is that, from the new_split function, if the data cannot be split anymore, the col column will be empty so we know to add this to the list
                    leaf.append(i)
                    f.append([0, 0, i[0], i[1][target].mean()])
                else:
                    t.append(0)
                    p.append(1)
                    new.append([t, s1])
                    new.append([p, s2])
                    f.append([col, sp, i[0]])
            except ValueError: #This was brought in to sort a weird error where the algortithm was only finding 4 values from the list when there was five
                leaf.append(i)
                f.append([0, 0, i[0], i[1][target].mean()])
        d=new
    for j in d:
        f.append([0, 0, j[0], j[1][target].mean()])
    d.extend(leaf)
    return d, f

In [7]:
def split_rf(dataframe, target, min_leaf, max_features):
    n=dataframe[target].unique().sum()
    variables=dataframe.drop(target, axis=1, inplace=False) #this drops the target column from the data for the next line
    n_cols=round(len(variables.columns)*max_features) #The number of random columns we will look at
    options=variables.columns.tolist() #A list of all column names in the data set
    select=random.sample(range(len(variables.columns)), n_cols) # how to randomly sel
    colls=[]
    for i in select:
        colls.append(options[i])
    #print('ncols={}'.format(n_cols))
    #print('options={}'.format(options))
    #print('select={}'.format(select))
    #print('the possible cols are{}'.format(colls))
    var=[variance(dataframe, target), 0,0,0, 0] # making an array to contain information about how the split is made
    for col in colls:
        if dataframe[col].dtypes=='int64' or 'float64':
            c=dataframe.sort_values(col) #This sorts each of the variable columns 
            u=c[col].unique()            
            for i in range(0, len(u)-1): #np.linspace(0, len(u)-1):
                split=(u[i]+u[i+1])/2
                l=len(dataframe)
                s1=pd.DataFrame(dataframe[dataframe[col]<split])
                s2=pd.DataFrame(dataframe[dataframe[col]>split])    
                v=variance(s1, target)*len(s1)/l+variance(s2, target)*len(s2)/l # This is where I introduced a weighting, outlined above
                if v<var[0] and len(s1)>min_leaf and len(s2)>min_leaf:
                    var=[v, col, split, s1, s2]
    if var[1]==0:
        var='leaf'
    return var

In [8]:
def random_forest(training_data, target, min_data, max_branches, max_features, n_trees):
    trees=[]
    for i in range(n_trees):
        data=bootstrap(training_data)
        #print(data)
        leaves, splits=rf_trees(data, target, min_data, max_branches, max_features)
        #print(splits)
        branch=plant(splits)
        print(branch)
        trees.append(branch)
    return trees

In [9]:
def plant(leaves):
    E=leaves
    max=len(E[-1][2])
    branch=[]
    for i in range(0, max+1):
        y=np.zeros(2**i).tolist()
        for j in E[1:]:
            if len(j[2])==i:
                n=0           
                for k in range(len(j[2])):
                    if j[2][k]==1:
                        n+=2**(len(j[2])-k-1)
                y[n]=j 
        branch.append(y)
    branch[0][0]=E[0]
    return branch

In [10]:
def bootstrap(train):
    boot=train.copy()
    n=len(boot)
    vals=np.random.randint(n, size=n)
    b=pd.DataFrame()
    for i in vals:
        b=pd.concat([b,boot.loc[i,:]], axis=1)
    b=b.T#.reset_index(drop=True)   ## Note sure if I need to reset the index for the algorithm to work or not.
    return b

In [11]:
def forest_predict(data, tree):
    n=0
    start=len(data.columns)
    for i in tree:
        name='prediction_{}'.format(str(n))
        n+=1
        data=predict(data, i, name)
    preds=data.iloc[:,start:]
    data['final_predictions']=preds.T.mean().round(1).tolist()
    return data